In [37]:
import plotly
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import pandas as pd

df = pd.read_csv('OGData.csv')
df = df[df["council_district"].notnull()]
df.dropna
df.head()

,Unnamed: 0,bl_id,building_name,address,utility,account,consump_month,consump_year,bge_kWh_used,bge_therms_used,...,block,lot,lat,lon,sq_ft,property_type,agency_occupied,year_built,neighborhood,council_district
0,0,B00035,Baltimore City Police Northern District,2201 West Cold Spring Lane,bge,3.275661e+09,12.0,2019.0,36551.0,1173.0,...,3357E,009A,39.34304,-76.65236,"25,508.47","Public Services, police",BCPD,2001.0,Woodberry,6.0
2,2,B00022,Baltimore City Mounted Police Unit,411 Fallsway St,bge,2.119590e+09,12.0,2018.0,7301.0,0.0,...,1255,2,39.29556,-76.61028,"11,270.59","Public Services, police",BCPD,NaN,Penn - Fallsway,11.0
3,3,B00035,Baltimore City Police Northern District,2201 West Cold Spring Lane,bge,3.275661e+09,11.0,2019.0,33586.0,1151.0,...,3357E,009A,39.34304,-76.65236,"25,508.47","Public Services, police",BCPD,2001.0,Woodberry,6.0
5,5,B00035,Baltimore City Police Northern District,2201 West Cold Spring Lane,bge,3.275661e+09,10.0,2019.0,38840.0,1430.0,...,3357E,009A,39.34304,-76.65236,"25,508.47","Public Services, police",BCPD,2001.0,Woodberry,6.0
7,7,B00182,Lower Park Heights Multi Purpose Building,3939 Reisterstown Rd,bge,1.198781e+09,7.0,2019.0,42814.0,0.0,...,3321,23,39.33373,-76.66336,"33,707","Public Assembly, Other",CAC,1978.0,Central Park Heights,6.0


In [97]:
property_type = {'Public Services, police': 1,'Education, vocational': 2,
                'Healthcare, Senior Care Community': 3, 'Healthcare, Urgent Care/Clinic/Out Patient': 4,
                'Lodging, Residential Care': 5, 'Office': 6, 'Public Assembly, Convention Center': 7,
                'Public Assembly, Museum': 8, 'Public Assembly, Other': 9, 'VACANT': 10,
                'Warehouse, self storage': 11} 
#df.property_type = [property_type[item] for item in df.property_type] 
agency_occupied = {'BCPD': 1, 'CAC': 2, 'Civic Centers': 3, 'DGS': 4, 'DGS HISTORIC': 5, 'DOT': 6,
                  'HEALTH': 7, 'HEALTH (BARCS)': 8, 'MOED': 9, 'MOHS': 10, 
                  'MULTI': 11, 'PABC': 12, 'VACANT': 13}
#df.agency_occupied = [agency_occupied[item] for item in df.agency_occupied] 
df['sq_ft'] = df['sq_ft'].str.replace(',', '')
df.head()

,Unnamed: 0,bl_id,building_name,address,utility,account,consump_month,consump_year,bge_kWh_used,bge_therms_used,...,block,lot,lat,lon,sq_ft,property_type,agency_occupied,year_built,neighborhood,council_district
0,0,B00035,Baltimore City Police Northern District,2201 West Cold Spring Lane,bge,3.275661e+09,12.0,2019.0,36551.0,1173.0,...,3357E,009A,39.34304,-76.65236,25508.47,1,BCPD,2001.0,Woodberry,6.0
2,2,B00022,Baltimore City Mounted Police Unit,411 Fallsway St,bge,2.119590e+09,12.0,2018.0,7301.0,0.0,...,1255,2,39.29556,-76.61028,11270.59,1,BCPD,NaN,Penn - Fallsway,11.0
3,3,B00035,Baltimore City Police Northern District,2201 West Cold Spring Lane,bge,3.275661e+09,11.0,2019.0,33586.0,1151.0,...,3357E,009A,39.34304,-76.65236,25508.47,1,BCPD,2001.0,Woodberry,6.0
5,5,B00035,Baltimore City Police Northern District,2201 West Cold Spring Lane,bge,3.275661e+09,10.0,2019.0,38840.0,1430.0,...,3357E,009A,39.34304,-76.65236,25508.47,1,BCPD,2001.0,Woodberry,6.0
7,7,B00182,Lower Park Heights Multi Purpose Building,3939 Reisterstown Rd,bge,1.198781e+09,7.0,2019.0,42814.0,0.0,...,3321,23,39.33373,-76.66336,33707,9,CAC,1978.0,Central Park Heights,6.0


In [54]:
df_2017 = df[df['consump_year']==2017]
df_2018 = df[df['consump_year']==2018]
df_2019 = df[df['consump_year']==2019]
df_train = pd.concat([df_2017, df_2018], axis=0)
df_val = df_2019
df_test = df_2019
print(df_train.shape)
print(df_val.shape)

(896, 22)
(263, 22)


In [98]:
df_train.describe()

,Unnamed: 0,account,consump_month,consump_year,bge_kWh_used,bge_therms_used,lat,lon,property_type,year_built,council_district
count,823.000000,8.230000e+02,823.000000,823.000000,823.000000,823.000000,823.000000,823.000000,823.000000,823.000000,823.000000
mean,812.274605,4.986811e+09,6.515188,2017.498177,38229.284326,688.874848,39.302151,-76.617003,6.358445,1944.663426,9.527339
std,294.888275,2.628790e+09,3.444655,0.500301,64945.145122,1620.783391,0.024091,0.025458,3.381871,54.200603,3.227983
min,305.000000,7.861510e+08,1.000000,2017.000000,0.000000,0.000000,39.252880,-76.685455,1.000000,1793.000000,1.000000
25%,558.500000,2.941411e+09,4.000000,2017.000000,5427.000000,0.000000,39.287760,-76.622230,4.000000,1945.000000,7.000000
50%,815.000000,4.751312e+09,7.000000,2017.000000,21532.000000,32.000000,39.292200,-76.610640,7.000000,1960.000000,11.000000
75%,1069.500000,6.948371e+09,10.000000,2018.000000,41037.500000,803.500000,39.313340,-76.606900,9.000000,1976.000000,12.000000
max,1324.000000,9.815941e+09,12.000000,2018.000000,555775.000000,14564.000000,39.353810,-76.547422,11.000000,2008.000000,14.000000


Linear Regression

In [55]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline

In [87]:
features = ['property_type', 'year_built', 'council_district', 'consump_month', 'sq_ft']
target = ['bge_kWh_used']

In [88]:
df_train = df_train[df_train["year_built"].notnull()]
m = LinearRegression()
m.fit(df_train[features],df_train[target])
m.score(df_train[features],df_train[target])

0.32272424639222397

In [89]:
df_val = df_val[df_val["year_built"].notnull()]
m.score(df_val[features],df_val[target])

0.31833278120827674

In [90]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

lm = LinearRegression()

poly = PolynomialFeatures(degree=2)

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


X_train_poly = poly.fit_transform(X_train)
X_val_poly = poly.transform(X_val)

lm_poly = LinearRegression()

In [91]:
new_cols = poly.get_feature_names(X_train.columns)

In [92]:
lm.fit(X_train, y_train)
print(f'Linear Regression val R^2: {lm.score(X_val, y_val):.3f}')

lm_poly.fit(X_train_poly, y_train)
print(f'Degree 2 polynomial regression val R^2: {lm_poly.score(X_val_poly, y_val):.3f}')

Linear Regression val R^2: 0.318
Degree 2 polynomial regression val R^2: 0.385


Lasso

In [93]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_val_scaled = scaler.transform(X_val_poly)

In [94]:
from sklearn.linear_model import LassoCV, Lasso
a =  2000
for i in range(20):
   lm_poly = Lasso(alpha=a, max_iter=100000, tol=0.001)
   lm_poly.fit(X_train_scaled,y_train)
   print(a,lm_poly.score(X_train_scaled, y_train),lm_poly.score(X_val_scaled, y_val))
   a+=50

2000 0.32903187122709243 0.3265432486060278
2050 0.32887466171867363 0.3264187601901141
2100 0.328712671149511 0.3262894972027196
2150 0.32854677754049744 0.32615596635830996
2200 0.3283769809122412 0.3260181676937516
2250 0.32820328128497034 0.3258761012452229
2300 0.3280246012885407 0.32572907388502104
2350 0.3278441680799373 0.3255791300006756
2400 0.3276587595814795 0.32542426090563903
2450 0.3274694431738897 0.3252650901866479
2500 0.3272762288708225 0.32510168632936637
2550 0.32707910675453855 0.3249339820823103
2600 0.32687674396183464 0.32476111122459683
2650 0.3266727803922106 0.3245855402782635
2700 0.3264638736909607 0.32440497955228775
2750 0.3262499734505164 0.3242193708622906
2800 0.3260349546999075 0.32403146543174366
2850 0.3258144161635924 0.3238381631936261
2900 0.32558997040564513 0.3236405664870833
2950 0.3253616221491695 0.3234387049078557


In [95]:
a =2500
lm_poly = Lasso(alpha=a, max_iter=100000, tol=0.001)
lm_poly.fit(X_train_scaled,y_train)
print(a,lm_poly.score(X_train_scaled, y_train),lm_poly.score(X_val_scaled, y_val))

2500 0.3272762288708225 0.32510168632936637


In [99]:
from sklearn.linear_model import LinearRegression
lrmodel=LinearRegression()
lrmodel.fit(df_train[features],df_train[target])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [102]:
predictedvalues = lrmodel.predict(df_val[features])
from sklearn.metrics import mean_squared_error
lrmodelrmse = np.sqrt(mean_squared_error(predictedvalues, df_val[target]))
print("RMSE value for Linear regression is", lrmodelrmse)

RMSE value for Linear regression is 51060.13670711602


In [103]:
from sklearn.ensemble import RandomForestRegressor
rfrmodel = RandomForestRegressor(n_estimators=100, random_state=101)

rfrmodel.fit(df_train[features],df_train[target])
rfrmodel_pred= rfrmodel.predict(df_val[features])
rfrmodel_rmse=np.sqrt(mean_squared_error(rfrmodel_pred, df_val[target]))
print("RMSE value for Random forest regression is ",rfrmodel_rmse)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RMSE value for Random forest regression is  44172.27019832995
